In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score

In [2]:
datasets = pd.read_csv('../datasets/train_preprocessed.csv').dropna()
X_train = datasets["content"].astype("string").to_numpy()
y_train = datasets["sentiment"].astype("category").cat.codes.to_numpy()

In [3]:
datasets_t = pd.read_csv('../datasets/test_preprocessed.csv').dropna()
X_test = datasets_t["content"].astype("string").to_numpy()
y_test = datasets_t["sentiment"].astype("category").cat.codes.to_numpy()

In [52]:
df = pd.read_csv("../datasets/tweet_emotions.csv")
x = df['content']
y = df['sentiment']
x, x_test, y, y_test = train_test_split(x,y, stratify=y, test_size=0.25, random_state=42)

In [5]:
x = pd.read_csv('../datasets/train_preprocessed.csv').content.to_numpy()
y = pd.read_csv('../datasets/train_preprocessed.csv').sentiment.to_numpy()
X_test = pd.read_csv('../datasets/test_preprocessed.csv').content.to_numpy()
y_test = pd.read_csv('../datasets/test_preprocessed.csv').sentiment.to_numpy()

In [5]:
vec = CountVectorizer(stop_words='english')
x = vec.fit_transform(X_train).toarray()
x_test = vec.transform(X_test).toarray()

In [6]:
model = MultinomialNB()
model.fit(x, y_train)

MultinomialNB()

In [7]:
model.score(x_test, y_test)

0.315773493244269

In [8]:
pred = model.predict((x_test))

In [9]:
f1 = f1_score(y_test, pred, average='weighted')
f1

0.2696385599978806